### PROJECT 1

Docelowy notebook w którym będzie finalny kod :-)

In [1]:
# imports
import torch.nn as nn
import torch.optim as optim
import numpy as np
from source.utils.config_manager import ConfigManager

# hyperparameters tuning
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from source.hyperparams_tuning import HyperparameteresTunner

# transformers
from source.transformers import TestTransformersFactory
from source.transformers import TrainTrasformersFactory

# data loaders
from source.dataloaders.cnn_data_loaders_factory import CNNDataLoaderFactory

# networks
from source.custom_cnn.pytorch_ray_tutorial_cnn import Net
from source.custom_cnn.dropout_batch_norm_cnn import DBN_cnn

In [2]:
dataset_name = 'cifar10'
dataset_path = ConfigManager.get_dataset_path(dataset_name)

config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD]),
    "lr": tune.choice([0.001]),
    "momentum": tune.choice([0.9])
}

config_net_1 = {
    "type": tune.choice([DBN_cnn]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1])
}

config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_kaggle())
    ]),
    "batch_size": tune.choice([64]),
    "epochs": 150,
    "net": tune.choice([config_net_1]),
    "tuning_id": "DBN"
}



In [3]:

gpus_per_trial = 2
max_num_epochs = 20
num_samples = 10
local_dir = ConfigManager.get_tuning_results_path()
    
config_criterion_1 = {
    "type": tune.choice([nn.CrossEntropyLoss])
}

config_optimizer_1 = {
    "type": tune.choice([optim.SGD]),
    "lr": tune.loguniform(1e-4, 1e-1),
}

config_net_1 = {
    "type": tune.choice([DBN_cnn]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config_net_2 = {
    "type": tune.choice([Net]),
    "optimizer": tune.choice([config_optimizer_1]),
    "criterion": tune.choice([config_criterion_1]),
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
}

config = {
    "data_loaders_factory": tune.choice([
        CNNDataLoaderFactory(dataset_path,
            TrainTrasformersFactory.get_transformer_kaggle(), TestTransformersFactory.get_transformer_kaggle())
    ]),
    "batch_size": tune.choice([2, 4, 8, 16]),
    "epochs": 20,
    "net": tune.choice([config_net_1]),
    "tuning_id": "XYZ"
}

scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2)

reporter = JupyterNotebookReporter(max_progress_rows=20, overwrite=True, max_report_frequency=10, max_error_rows=5)

In [4]:
tuner = HyperparameteresTunner()

In [5]:
tuner.tune(dataset_name, local_dir, config, scheduler, reporter, num_samples=4)

KeyboardInterrupt: 